# Top 1000 episodes on IMDb

Run the code below to go into previous directory

In [1]:
%cd ..

C:\Users\ganiy\OneDrive\Documents\IMDB-TV


Uncomment the codes below and run to create the directories if this is your first time running it. If it returns a statement stating that the directories already exists, proceed.

In [2]:
#%mkdir anime-series-data/
#%mkdir anime-series-data-named/
#%mkdir cumulative-data/

In [3]:
import pandas as pd
import numpy as np
import os
%matplotlib inline
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import requests
import re
from requests import get

### Top 1000 episodes on IMDb list

In [4]:
# url with the link to the top 1000 episodes list
url = 'https://www.imdb.com/search/title/?title_type=tv_episode&num_votes=1000,&sort=user_rating,desc'
response = get(url)
episode_soup = BeautifulSoup(response.text, 'html.parser')
episode_box = episode_soup.find_all('div', class_ = "lister-item-content")
episode_list = []
for episode in episode_box:
    tv_show = episode.a.text.strip()
    episode_name = episode.find_all("a")[1].string
    ratings = episode.find("strong").text
    year = episode.find_all("span", class_ = "lister-item-year text-muted unbold")[1].text
    year = year.replace("(", "")
    year = year.replace(")", "")
    votes = episode.find("p", class_ = "sort-num_votes-visible")
    votes = votes.find_all("span")[1].text
    votes = votes.replace(",", "")
    desc = episode.find_all("p", class_ = "text-muted")[1].text.strip("\n")
    genre = episode.find_all("p", class_ = "text-muted")[0]
    genre = genre.find("span", class_ = "genre").text.strip("\n").strip(" ")
    #genre = genre.span["genre"]
    encoded_title = episode.a["href"]
    encoded_title = encoded_title.replace("/title/", "")
    encoded_title = encoded_title.replace("/", "")
    episode_list.append([tv_show, episode_name, ratings, year, votes, desc, genre, encoded_title])
n = 51
while n < 1000:
    # url with the link to the top 50 anime list
    url = 'https://www.imdb.com/search/title/?title_type=tv_episode&num_votes=1000,&sort=user_rating,desc&start=' + str(n) + '&ref_=adv_nxt'
    response = get(url)
    episode_soup = BeautifulSoup(response.text, 'html.parser')
    episode_box = episode_soup.find_all('div', class_ = "lister-item-content")
    for episode in episode_box:
        tv_show = episode.a.text.strip()
        episode_name = episode.find_all("a")[1].string
        ratings = episode.find("strong").text
        year = episode.find_all("span", class_ = "lister-item-year text-muted unbold")[1].text
        year = year.replace("(", "")
        year = year.replace(")", "")
        votes = episode.find("p", class_ = "sort-num_votes-visible")
        votes = votes.find_all("span")[1].text
        votes = votes.replace(",", "")
        desc = episode.find_all("p", class_ = "text-muted")[1].text.strip("\n")
        genre = episode.find_all("p", class_ = "text-muted")[0]
        genre = genre.find("span", class_ = "genre").text.strip("\n").strip(" ")
        encoded_title = episode.a["href"]
        encoded_title = encoded_title.replace("/title/", "")
        encoded_title = encoded_title.replace("/", "")
        episode_list.append([tv_show, episode_name, ratings, year, votes, desc, genre, encoded_title])
    n += 50

In [5]:
df = pd.DataFrame(episode_list, columns = ["tv_show", "episode", "rating", "year", "total_votes", "desc", "genre", "encoded_title"])

In [6]:
df["year"] = df["year"].str.replace("I 2019", "2019")
df["year"] = df["year"].astype(int)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   tv_show        1000 non-null   object
 1   episode        1000 non-null   object
 2   rating         1000 non-null   object
 3   year           1000 non-null   int32 
 4   total_votes    1000 non-null   object
 5   desc           1000 non-null   object
 6   genre          1000 non-null   object
 7   encoded_title  1000 non-null   object
dtypes: int32(1), object(7)
memory usage: 58.7+ KB


In [8]:
df.describe()

,year
count,1000.000000
mean,2014.628000
std,6.147634
min,1975.000000
25%,2012.000000
50%,2016.000000
75%,2019.000000
max,2022.000000


In [9]:
df.head()

,tv_show,episode,rating,year,total_votes,desc,genre,encoded_title
0,Breaking Bad,Ozymandias,10.0,2013,177058,Walt goes on the run. Jesse is taken hostage. ...,"Crime, Drama, Thriller",tt0903747
1,Attack on Titan,Hero,9.9,2019,94079,As Erwin's heroic charge buys Levi time to con...,"Animation, Action, Adventure",tt2560140
2,Attack on Titan,Perfect Game,9.9,2019,52124,"While one front is rained on by flames, the ot...","Animation, Action, Adventure",tt2560140
3,Star Wars: The Clone Wars,Victory and Death,9.9,2020,13848,Ahsoka and Rex must use their wit and skills t...,"Animation, Action, Adventure",tt0458290
4,Star Wars: The Clone Wars,The Phantom Apprentice,9.9,2020,13350,Ahsoka leads Republic clones to confront Maul'...,"Animation, Action, Adventure",tt0458290


In [10]:
df.tail()

,tv_show,episode,rating,year,total_votes,desc,genre,encoded_title
995,NCIS,Keep Going,9.2,2017,1119,"During a hit-and-run investigation, Palmer tri...","Action, Crime, Drama",tt0364845
996,It's Always Sunny in Philadelphia,Time's Up for the Gang,9.2,2018,3160,Paddy's has been put on a list of bars that ar...,Comedy,tt0472954
997,Adventure Time,Islands Part 5: Hide and Seek,9.2,2017,1247,Susan strong has flashbacks to her life when s...,"Animation, Action, Adventure",tt1305826
998,Code Black,The Devil's Workshop,9.2,2017,1147,A viral outbreak jeopardizes the doctors and p...,Drama,tt4452630
999,JoJo's Bizarre Adventure,"Crazy Diamond Is Unbreakable, Part 2",9.2,2016,1211,The battle between Josuke and Kira continues,"Animation, Action, Adventure",tt2359704


In [11]:
genres = df["genre"].str.split(", ", expand = True)
df["genre1"] = genres[0]
df["genre2"] = genres[1]
df["genre3"] = genres[2]
df.head()

,tv_show,episode,rating,year,total_votes,desc,genre,encoded_title,genre1,genre2,genre3
0,Breaking Bad,Ozymandias,10.0,2013,177058,Walt goes on the run. Jesse is taken hostage. ...,"Crime, Drama, Thriller",tt0903747,Crime,Drama,Thriller
1,Attack on Titan,Hero,9.9,2019,94079,As Erwin's heroic charge buys Levi time to con...,"Animation, Action, Adventure",tt2560140,Animation,Action,Adventure
2,Attack on Titan,Perfect Game,9.9,2019,52124,"While one front is rained on by flames, the ot...","Animation, Action, Adventure",tt2560140,Animation,Action,Adventure
3,Star Wars: The Clone Wars,Victory and Death,9.9,2020,13848,Ahsoka and Rex must use their wit and skills t...,"Animation, Action, Adventure",tt0458290,Animation,Action,Adventure
4,Star Wars: The Clone Wars,The Phantom Apprentice,9.9,2020,13350,Ahsoka leads Republic clones to confront Maul'...,"Animation, Action, Adventure",tt0458290,Animation,Action,Adventure


In [12]:
df.to_csv("cumulative-data/top1000_episodes_imdb.csv", index = False)

In [13]:
df

,tv_show,episode,rating,year,total_votes,desc,genre,encoded_title,genre1,genre2,genre3
0,Breaking Bad,Ozymandias,10.0,2013,177058,Walt goes on the run. Jesse is taken hostage. ...,"Crime, Drama, Thriller",tt0903747,Crime,Drama,Thriller
1,Attack on Titan,Hero,9.9,2019,94079,As Erwin's heroic charge buys Levi time to con...,"Animation, Action, Adventure",tt2560140,Animation,Action,Adventure
2,Attack on Titan,Perfect Game,9.9,2019,52124,"While one front is rained on by flames, the ot...","Animation, Action, Adventure",tt2560140,Animation,Action,Adventure
3,Star Wars: The Clone Wars,Victory and Death,9.9,2020,13848,Ahsoka and Rex must use their wit and skills t...,"Animation, Action, Adventure",tt0458290,Animation,Action,Adventure
4,Star Wars: The Clone Wars,The Phantom Apprentice,9.9,2020,13350,Ahsoka leads Republic clones to confront Maul'...,"Animation, Action, Adventure",tt0458290,Animation,Action,Adventure
...,...,...,...,...,...,...,...,...,...,...,...
995,NCIS,Keep Going,9.2,2017,1119,"During a hit-and-run investigation, Palmer tri...","Action, Crime, Drama",tt0364845,Action,Crime,Drama
996,It's Always Sunny in Philadelphia,Time's Up for the Gang,9.2,2018,3160,Paddy's has been put on a list of bars that ar...,Comedy,tt0472954,Comedy,None,None
997,Adventure Time,Islands Part 5: Hide and Seek,9.2,2017,1247,Susan strong has flashbacks to her life when s...,"Animation, Action, Adventure",tt1305826,Animation,Action,Adventure
998,Code Black,The Devil's Workshop,9.2,2017,1147,A viral outbreak jeopardizes the doctors and p...,Drama,tt4452630,Drama,None,None
